##### Traffic sign Dataset에서 전체 클래스 확인

In [6]:
import json
import pprint

with open('D:/traffic_sign/traffic_sign_json/886658_614.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

pprint.pprint(json_data)

{'annotations': [{'area': 8352,
                  'bbox': [1471, 1187, 87, 96],
                  'category_id': 455,
                  'id': 'df1f9c73-a57c-4807-b172-8ef17acef2de',
                  'image_id': '886658_614',
                  'isSmallType': 'Y',
                  'segmentation': [{'x': 1555, 'y': 1187},
                                   {'x': 1474, 'y': 1189},
                                   {'x': 1471, 'y': 1197},
                                   {'x': 1472, 'y': 1280},
                                   {'x': 1494, 'y': 1283},
                                   {'x': 1556, 'y': 1279},
                                   {'x': 1558, 'y': 1250},
                                   {'x': 1555, 'y': 1187}]},
                 {'area': 7224,
                  'bbox': [1470, 1102, 86, 84],
                  'category_id': 404,
                  'id': '42b2a1cc-acfe-4d60-9b35-3500cc803653',
                  'image_id': '886658_614',
                  'isSmallType': 'Y'

In [7]:
for i in range(len(json_data['categories'])):
    code = json_data['categories'][i]['class_id']
    name = json_data['categories'][i]['class_name']
    print(code, name)

401 十자형 교차로
406 T자형 교차로
411 Y자형 교차로
416 ㅏ자형 교차로
421 ㅓ자형 교차로
426 철도 건널목
431 오르막경사
436 내리막경사
441 신호기
446 미끄럼 도로
402 강변도로(추락주의)
407 과속 방지턱
412 낙석주의
417 어린이 보호
422 자전거
427 도로 공사중
432 터널
437 야생동물 보호
442 위험
447 상습 정체구간
403 통행금지
408 화물자동차 통행금지
413 이륜자동차 통행금지
418 자전거 통행금지
423 진입금지
428 직진금지
433 우회전금지
438 좌회전금지
443 유턴금지
448 앞지르기금지
404 주차금지/주정차금지
409 최고속도제한
414 서행(천천히)
419 정지(STOP)
424 양보
429 보행자 보행금지
434 자동차전용도로
439 자전거전용도로
444 자전거 및 보행자 겸용도로
449 회전교차로
405 직진
410 우회전
415 좌회전
420 유턴
425 자전거및보행자통행구분
430 자전거전용차로
435 보행자전용도로
440 횡단보도
445 노인보호구역
450 어린이보호구역
451 장애인보호구역
452 일방통행
453 비보호좌회전
454 버스전용차로
455 견인지역
456 경음기사용금지


##### Traffic sign Dataset에서 필요한 클래스만 추출하는 코드
- 탐지하고자 하는 class_id:
    - 404 주차금지/주정차금지
    - 450 어린이보호구역

In [8]:
import os
import json

# 탐지하고자 하는 category_id 리스트
code_to_detect = [404, 450]

# 경로 설정
label_src_path = 'D:/traffic_sign/traffic_sign_json'

In [5]:
# 특정 카테고리 ID를 가진 annotation 필터링
def filter_annotations(data, categories):
    return [annotation for annotation in data['annotations'] if annotation['category_id'] in categories]

In [6]:
# 특정 카테고리 ID를 가진 annotation이 포함된 파일 이름들을 필터링하여 반환
def process_files(filenames, src_label_path, categories, encoding='utf-8'):
    valid_files = []
    for filename in filenames:
        json_file_path = os.path.join(src_label_path, filename)
        
        # JSON 파일 열기
        with open(json_file_path, 'r', encoding=encoding) as json_data:
            data = json.load(json_data)
        
        # 탐지하고자 하는 category_id가 아닌 annotation 제거하기
        filtered_annotations = filter_annotations(data, categories)

        # 유효한 annotation이 있는 경우 파일 이름 저장
        if filtered_annotations:
            valid_files.append(filename)

    return valid_files

In [7]:
# 유효한 파일 이름을 기반으로 JSON 파일을 보존하고, 나머지 JSON 파일을 삭제
def retain_valid_json_files(valid_files, src_label_path):
    # JSON 디렉토리에서 모든 JSON 파일 이름을 가져오기
    all_json_files = os.listdir(src_label_path)

    for json_filename in all_json_files:
        json_file_path = os.path.join(src_label_path, json_filename)
        # JSON 파일이 유효한 파일 이름 목록에 없는 경우 삭제
        if json_filename not in valid_files:
            os.remove(json_file_path)
            print(f'{json_file_path} JSON 파일이 삭제되었습니다.')
        else:
            print(f'{json_file_path} JSON 파일이 보존되었습니다.')

In [8]:
image_src_path = 'D:/traffic_sign/traffic_sign_image'

In [9]:
# 유효한 파일 이름을 기반으로 이미지 파일을 보존하고, 나머지 이미지 파일을 삭제
def retain_valid_image_files(valid_files, src_image_path):
    # 이미지 디렉토리에서 모든 이미지 파일 이름을 가져오기
    all_image_files = os.listdir(src_image_path)

    # 유효한 파일 이름 목록에서 확장자를 제거하여 이미지 파일 이름을 생성
    valid_image_filenames = [os.path.splitext(filename)[0] + '.jpg' for filename in valid_files]

    for image_filename in all_image_files:
        image_file_path = os.path.join(src_image_path, image_filename)
        # 이미지 파일이 유효한 파일 이름 목록에 없는 경우 삭제
        if image_filename not in valid_image_filenames:
            os.remove(image_file_path)
            print(f'{image_file_path} 이미지 파일이 삭제되었습니다.')
        else:
            print(f'{image_file_path} 이미지 파일이 보존되었습니다.')

In [10]:
def main():
    filenames = os.listdir(label_src_path)  # JSON 파일 리스트
    filenames = [filename for filename in filenames if filename.endswith('.json')]

    valid_files = process_files(filenames, label_src_path, code_to_detect, encoding='utf-8')

    # 유효한 파일 이름과 동일한 이미지 파일 제외하고 나머지 이미지 파일 삭제
    retain_valid_image_files(valid_files, image_src_path)

    # 유효한 파일 이름과 동일한 JSON 파일 제외하고 나머지 JSON 파일 삭제
    retain_valid_json_files(valid_files, label_src_path)

if __name__ == "__main__":
    valid_files = main()

D:/표지판/traffic_sign_image\1001015_614.jpg 이미지 파일이 보존되었습니다.
D:/표지판/traffic_sign_image\1001021_614.jpg 이미지 파일이 삭제되었습니다.
D:/표지판/traffic_sign_image\1001026_614.jpg 이미지 파일이 보존되었습니다.
D:/표지판/traffic_sign_image\1001031_614.jpg 이미지 파일이 보존되었습니다.
D:/표지판/traffic_sign_image\1001039_614.jpg 이미지 파일이 보존되었습니다.
D:/표지판/traffic_sign_image\1001045_614.jpg 이미지 파일이 삭제되었습니다.
D:/표지판/traffic_sign_image\1001070_614.jpg 이미지 파일이 보존되었습니다.
D:/표지판/traffic_sign_image\1001094_614.jpg 이미지 파일이 보존되었습니다.
D:/표지판/traffic_sign_image\1001166_614.jpg 이미지 파일이 보존되었습니다.
D:/표지판/traffic_sign_image\1001175_614.jpg 이미지 파일이 보존되었습니다.
D:/표지판/traffic_sign_image\1001230_614.jpg 이미지 파일이 보존되었습니다.
D:/표지판/traffic_sign_image\1002783_614.jpg 이미지 파일이 보존되었습니다.
D:/표지판/traffic_sign_image\1002795_614.jpg 이미지 파일이 보존되었습니다.
D:/표지판/traffic_sign_image\1002803_614.jpg 이미지 파일이 보존되었습니다.
D:/표지판/traffic_sign_image\1002816_614.jpg 이미지 파일이 보존되었습니다.
D:/표지판/traffic_sign_image\1002818_614.jpg 이미지 파일이 보존되었습니다.
D:/표지판/traffic_sign_image\1002877_614.jpg 이미지 파일이 삭제되었습니

In [11]:
import glob
 
folder_path = 'D:/traffic_sign/traffic_sign_json'
file_count = len(glob.glob(folder_path + '/*'))
print(file_count)

5112


In [12]:
import glob
 
folder_path = 'D:/traffic_sign/traffic_sign_image'
file_count = len(glob.glob(folder_path + '/*'))
print(file_count)

5112


##### annotation 정리

In [1]:
import json
import pprint

with open('D:/traffic_sign/traffic_sign_json/886658_614.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

pprint.pprint(json_data)

{'annotations': [{'area': 8352,
                  'bbox': [1471, 1187, 87, 96],
                  'category_id': 455,
                  'id': 'df1f9c73-a57c-4807-b172-8ef17acef2de',
                  'image_id': '886658_614',
                  'isSmallType': 'Y',
                  'segmentation': [{'x': 1555, 'y': 1187},
                                   {'x': 1474, 'y': 1189},
                                   {'x': 1471, 'y': 1197},
                                   {'x': 1472, 'y': 1280},
                                   {'x': 1494, 'y': 1283},
                                   {'x': 1556, 'y': 1279},
                                   {'x': 1558, 'y': 1250},
                                   {'x': 1555, 'y': 1187}]},
                 {'area': 7224,
                  'bbox': [1470, 1102, 86, 84],
                  'category_id': 404,
                  'id': '42b2a1cc-acfe-4d60-9b35-3500cc803653',
                  'image_id': '886658_614',
                  'isSmallType': 'Y'

In [2]:
# categories 종류와 이름 추출
codelist = []
namelist = []

for i in range(len(json_data['categories'])):
    codelist.append(json_data['categories'][i]['class_id'])
    namelist.append(json_data['categories'][i]['class_name'])

# code와 name을 1:1 매칭
mapping_dict = list(zip(codelist, namelist))

print(mapping_dict)

[(401, '十자형 교차로'), (406, 'T자형 교차로'), (411, 'Y자형 교차로'), (416, 'ㅏ자형 교차로'), (421, 'ㅓ자형 교차로'), (426, '철도 건널목'), (431, '오르막경사'), (436, '내리막경사'), (441, '신호기'), (446, '미끄럼 도로'), (402, '강변도로(추락주의)'), (407, '과속 방지턱'), (412, '낙석주의'), (417, '어린이 보호'), (422, '자전거'), (427, '도로 공사중'), (432, '터널'), (437, '야생동물 보호'), (442, '위험'), (447, '상습 정체구간'), (403, '통행금지'), (408, '화물자동차 통행금지'), (413, '이륜자동차 통행금지'), (418, '자전거 통행금지'), (423, '진입금지'), (428, '직진금지'), (433, '우회전금지'), (438, '좌회전금지'), (443, '유턴금지'), (448, '앞지르기금지'), (404, '주차금지/주정차금지'), (409, '최고속도제한'), (414, '서행(천천히)'), (419, '정지(STOP)'), (424, '양보'), (429, '보행자 보행금지'), (434, '자동차전용도로'), (439, '자전거전용도로'), (444, '자전거 및 보행자 겸용도로'), (449, '회전교차로'), (405, '직진'), (410, '우회전'), (415, '좌회전'), (420, '유턴'), (425, '자전거및보행자통행구분'), (430, '자전거전용차로'), (435, '보행자전용도로'), (440, '횡단보도'), (445, '노인보호구역'), (450, '어린이보호구역'), (451, '장애인보호구역'), (452, '일방통행'), (453, '비보호좌회전'), (454, '버스전용차로'), (455, '견인지역'), (456, '경음기사용금지')]


In [12]:
def sign_convert_via(js):
    dic = {}
    dic['fileref'] = ""
    dic['size'] = js['images'][0]['width'] * js['images'][0]['height']
    dic['filename'] = js['images'][0]['file_name']
    dic['base64_img_data'] = ""
    dic['file_attributes'] = {}
    dic['regions'] = {}  # 한 이미지에서 탐지되는 클래스 수

    # 찾고자 하는 annotation이 아니면 삭제
    annotations_to_keep = [annotation for annotation in js['annotations'] if annotation['category_id'] in code_to_detect]
    js['annotations'] = annotations_to_keep

    for i, annotation in enumerate(js['annotations']):
        # segmentation 안에 있는 x, y값을 저장하기 위한 리스트 작성
        x_list = []
        y_list = []

        # 각각의 annotation 값에 대한 segmentation의 x, y 값 저장
        for point in annotation['segmentation']:
            x_list.append(point['x'])
            y_list.append(point['y'])

        # 어떤 카테고리인지 확인
        categories = js['annotations'][i]['category_id']
        cate_name = None
        for k in range(len(mapping_dict)):
            if mapping_dict[k][0] == categories:
                cate_name = mapping_dict[k][1]
                break

        if cate_name:
            dic['regions'][i] = {
                'region_attributes': {
                    'label': cate_name
                },
                'shape_attributes': {
                    "name": 'polygon',
                    "all_points_x": x_list,
                    "all_points_y": y_list
                }
            }

    return dic

In [13]:
import json
with open('D:/traffic_sign/traffic_sign_json/886658_614.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [14]:
sign_convert_via(data)

{'fileref': '',
 'size': 0,
 'filename': '886658_614.jpg',
 'base64_img_data': '',
 'file_attributes': {},
 'regions': {0: {'region_attributes': {'label': '주차금지/주정차금지'},
   'shape_attributes': {'name': 'polygon',
    'all_points_x': [1506,
     1492,
     1476,
     1470,
     1470,
     1479,
     1489,
     1497,
     1521,
     1538,
     1550,
     1556,
     1556,
     1540,
     1526,
     1506],
    'all_points_y': [1102,
     1108,
     1124,
     1136,
     1160,
     1175,
     1183,
     1186,
     1186,
     1178,
     1165,
     1150,
     1128,
     1108,
     1102,
     1102]}}}}

##### 각 label 별로 json 및 image 나누기

In [15]:
import shutil

def process_json_files_in_directory(json_dir, image_dir, output_dir):
    for filename in os.listdir(json_dir):
        if filename.endswith('.json'):
            json_file_path = os.path.join(json_dir, filename)
            image_file_path = os.path.join(image_dir, filename.replace('.json', '.jpg'))
            
            if os.path.exists(image_file_path):
                with open(json_file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    result = sign_convert_via(data)
                
                labels = [result['regions'][i]['region_attributes']['label'] for i in range(len(result['regions']))]

                if "주차금지/주정차금지" in labels:
                    type_name = "주차금지_주정차금지"
                elif "어린이보호구역" in labels:
                    type_name = "어린이보호구역"
                else:
                    type_name = "unknown"

                type_dir = os.path.join(output_dir, type_name)
                os.makedirs(type_dir, exist_ok=True)

                # JSON 및 이미지 파일을 저장할 서브 디렉터리 생성
                json_output_dir = os.path.join(type_dir, 'json')
                image_output_dir = os.path.join(type_dir, 'image')
                os.makedirs(json_output_dir, exist_ok=True)
                os.makedirs(image_output_dir, exist_ok=True)

                # JSON 파일 복사
                json_output_path = os.path.join(json_output_dir, os.path.basename(json_file_path))
                shutil.copy2(json_file_path, json_output_path)

                # 이미지 파일 복사
                image_output_path = os.path.join(image_output_dir, os.path.basename(image_file_path))
                shutil.copy2(image_file_path, image_output_path)

                print(f"Processed {filename}: Type - {type_name}")
            else:
                print(f"Image file for {filename} not found.")

In [41]:
json_dir = "D:/traffic_sign/traffic_sign_json"
image_dir = "D:/traffic_sign/traffic_sign_image"
output_dir = "D:/traffic_sign"

process_json_files_in_directory(json_dir, image_dir, output_dir)

Processed 1001015_614.json: Type - 주차금지_주정차금지
Processed 1001026_614.json: Type - 주차금지_주정차금지
Processed 1001031_614.json: Type - 주차금지_주정차금지
Processed 1001039_614.json: Type - 주차금지_주정차금지
Processed 1001070_614.json: Type - 주차금지_주정차금지
Processed 1001094_614.json: Type - 주차금지_주정차금지
Processed 1001166_614.json: Type - 주차금지_주정차금지
Processed 1001175_614.json: Type - 주차금지_주정차금지
Processed 1001230_614.json: Type - 주차금지_주정차금지
Processed 1002783_614.json: Type - 주차금지_주정차금지
Processed 1002795_614.json: Type - 주차금지_주정차금지
Processed 1002803_614.json: Type - 주차금지_주정차금지
Processed 1002816_614.json: Type - 주차금지_주정차금지
Processed 1002818_614.json: Type - 주차금지_주정차금지
Processed 1002926_614.json: Type - 어린이보호구역
Processed 1003000_614.json: Type - 주차금지_주정차금지
Processed 1003002_614.json: Type - 주차금지_주정차금지
Processed 1003013_614.json: Type - 주차금지_주정차금지
Processed 1003018_614.json: Type - 주차금지_주정차금지
Processed 1003032_614.json: Type - 주차금지_주정차금지
Processed 1003048_614.json: Type - 주차금지_주정차금지
Processed 1003701_614.json: Type - 주차

##### 랜덤으로 300개 정도의 데이터 추출

In [23]:
import random

def select_random_files(source_dir, destination_dir, num_files=300):
    files = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]
    
    # 파일 수가 num_files보다 적으면 전체 파일을 사용
    if len(files) < num_files:
        print(f"파일 수가 {num_files}보다 적으므로 전체 파일을 사용합니다.")
        num_files = len(files)
    
    # 랜덤으로 파일 선택
    selected_files = random.sample(files, num_files)
    os.makedirs(destination_dir, exist_ok=True)
    
    # 파일 복사
    for file in selected_files:
        shutil.copy2(os.path.join(source_dir, file), os.path.join(destination_dir, file))
    
    print(f"{num_files}개의 파일이 {destination_dir}에 복사되었습니다.")

In [44]:
source_dir = "D:/traffic_sign/주차금지_주정차금지/image"
destination_dir = "D:/traffic_sign/주차금지_주정차금지/random/image"

select_random_files(source_dir, destination_dir)

300개의 파일이 D:/traffic_sign/주차금지_주정차금지/random/image에 복사되었습니다.


In [45]:
source_dir = "D:/traffic_sign/어린이보호구역/image"
destination_dir = "D:/traffic_sign/어린이보호구역/random/image"

select_random_files(source_dir, destination_dir)

300개의 파일이 D:/traffic_sign/어린이보호구역/random/image에 복사되었습니다.


##### train / val 나누기

In [24]:
def split_data_into_train_val(source_dir, train_dir, val_dir, train_ratio=0.8):
    # 파일 리스트 가져오기
    files = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]
    
    random.shuffle(files)
    
    # 파일을 train_ratio 비율로 나누기
    train_size = int(len(files) * train_ratio)
    train_files = files[:train_size]
    val_files = files[train_size:]
    
    # train 및 val 디렉터리 생성
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    
    for file in train_files:
        shutil.copy2(os.path.join(source_dir, file), os.path.join(train_dir, file))
    
    for file in val_files:
        shutil.copy2(os.path.join(source_dir, file), os.path.join(val_dir, file))
    
    print(f"{len(train_files)}개의 파일이 {train_dir}에 복사되었습니다.")
    print(f"{len(val_files)}개의 파일이 {val_dir}에 복사되었습니다.")

In [25]:
source_dir = "D:/traffic_sign/주차금지_주정차금지/random/image"
train_dir = "D:/traffic_sign/주차금지_주정차금지/random/train"
val_dir = "D:/traffic_sign/주차금지_주정차금지/random/val"

split_data_into_train_val(source_dir, train_dir, val_dir)

196개의 파일이 D:/traffic_sign/주차금지_주정차금지/random/train에 복사되었습니다.
50개의 파일이 D:/traffic_sign/주차금지_주정차금지/random/val에 복사되었습니다.


In [26]:
source_dir = "D:/traffic_sign/어린이보호구역/random/image"
train_dir = "D:/traffic_sign/어린이보호구역/random/train"
val_dir = "D:/traffic_sign/어린이보호구역/random/val"

split_data_into_train_val(source_dir, train_dir, val_dir)

186개의 파일이 D:/traffic_sign/어린이보호구역/random/train에 복사되었습니다.
47개의 파일이 D:/traffic_sign/어린이보호구역/random/val에 복사되었습니다.


##### 각 라벨에 맞는 json 취합

In [27]:
def combine_json_files(div_dir, json_source_dir, output_json_path):
    combined_data = []

    # 디렉터리에서 이미지 파일 리스트 가져오기
    image_files = [f for f in os.listdir(div_dir) if os.path.isfile(os.path.join(div_dir, f)) and f.endswith(('.jpg', '.jpeg', '.png'))]

    for image_file in image_files:
        json_file_name = image_file.rsplit('.', 1)[0] + '.json'  # 이미지 파일명에서 확장자 제거하고 .json 확장자 붙이기
        json_file_path = os.path.join(json_source_dir, json_file_name)

        if os.path.exists(json_file_path):
            with open(json_file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                converted_data = sign_convert_via(data)
                combined_data.append(converted_data)
        else:
            print(f"JSON 파일을 찾을 수 없습니다: {json_file_name}")

    # 결합된 JSON 데이터를 하나의 파일로 저장
    with open(output_json_path, 'w', encoding='utf-8') as output_file:
        json.dump(combined_data, output_file, ensure_ascii=False, indent=4)

    print(f"결합된 JSON 데이터가 {output_json_path}에 저장되었습니다.")

In [28]:
# train json 병합
train_dir = "D:/traffic_sign/주차금지_주정차금지/random/train"
json_source_dir = "D:/traffic_sign/주차금지_주정차금지/json"
output_json_path = "D:/traffic_sign/주차금지_주정차금지/random/train_combined.json"

combine_json_files(train_dir, json_source_dir, output_json_path)

결합된 JSON 데이터가 D:/traffic_sign/주차금지_주정차금지/random/train_combined.json에 저장되었습니다.


In [29]:
# train json 병합
train_dir = "D:/traffic_sign/어린이보호구역/random/train"
json_source_dir = "D:/traffic_sign/어린이보호구역/json"
output_json_path = "D:/traffic_sign/어린이보호구역/random/train_combined.json"

combine_json_files(train_dir, json_source_dir, output_json_path)

결합된 JSON 데이터가 D:/traffic_sign/어린이보호구역/random/train_combined.json에 저장되었습니다.


In [30]:
# val json 병합
val_dir = "D:/traffic_sign/주차금지_주정차금지/random/val"
json_source_dir = "D:/traffic_sign/주차금지_주정차금지/json"
output_json_path = "D:/traffic_sign/주차금지_주정차금지/random/val_combined.json"

combine_json_files(val_dir, json_source_dir, output_json_path)

결합된 JSON 데이터가 D:/traffic_sign/주차금지_주정차금지/random/val_combined.json에 저장되었습니다.


In [31]:
# val json 병합
val_dir = "D:/traffic_sign/어린이보호구역/random/val"
json_source_dir = "D:/traffic_sign/어린이보호구역/json"
output_json_path = "D:/traffic_sign/어린이보호구역/random/val_combined.json"

combine_json_files(val_dir, json_source_dir, output_json_path)

결합된 JSON 데이터가 D:/traffic_sign/어린이보호구역/random/val_combined.json에 저장되었습니다.
